## BHMS3323 Practices of FinTech

## Lab8 - Build your own blockchain 2

In lab 7, we built the skeleton code of blockchain.

Our Blockchain stores transaction data and has the following characteristics:

1. New transactions will be placed into the memory pool (MEMPOOL).
2. Transaction data will be extracted from the MEMPOOL and be included in a new block.
3. The new block contains the information of transaction data and the previous block's hash value.
4. New block will be appended in the blockchain without any checking. 

Below are the skeleton code developed in the previous lesson:


In [1]:
import time
import json
import hashlib
import datetime

class BlockChain:
    def __init__(self):
        self.blocks = []  # empty chain
        self.mem_pool = [] # empty list
        
        ## add Genesis block
        self.add_new_block(100, "Genesis Block")
        
        
    def new_transaction(self, sender, recipient, amount):
        transaction = {
            "Sender": sender,
            "Recipient": recipient,
            "Amount(HKD)": amount,
            "Datetime": time.time()
        }
        self.mem_pool.append(transaction)
    
    def add_new_block(self, nounce, previous_hash=None):
        block = {
            'index': len(self.blocks) + 1,  # the next value of index
            'timestamp': time.time(),
            'transactions': self.mem_pool,
            'nounce': nounce,
            'previous_hash': previous_hash or self.hash(self.blocks[-1]),
        }
        
        self.mem_pool = [] # clear the pool
        self.blocks.append(block) # append the block to the chain
        
        return block
    
    def hash(self, block):
        data = json.dumps(block, sort_keys=True)  # convert the json object to string
        return hashlib.sha256(data.encode()).hexdigest()
        
    def print(self):
        for block in self.blocks:
            print("Block {}:".format(block['index']))
            print("Createion time: {}".format( self.convert_time(block['timestamp'])))
            print("Previous hash: {}".format(block['previous_hash']))
            print("Hash:{}".format(self.hash(block)))

            for transaction in block["transactions"]:
                print(transaction)

            print("")
        
    def convert_time(self, timestamp):
        return datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")
        

## Test the blockchain

Now, let's test our blockchain by building 2 blocks containing 6 transactions.

### 1st Block
- The first block in the blockchain is the genesis block in creation; the genesis block contains no useful information.

### 2nd block
- The second block contains two transactions:
1. Ray sends Joe \$200
2. Peter sends Bill \$400

### 3rd block
- The third block contains two transactions:
1. Christine sends Ray \$300
2. Peter sends Ray \$500
3. Ray sends Amy \$100
4. Peter sends Bill \$200


In [2]:
# Test your blockchain

blockchain = BlockChain()

# Add the transaction in the MEM_POOL.
blockchain.new_transaction("Ray", "Joe", 200)
blockchain.new_transaction("Peter", "Bill", 400)

# Create a new block and append the block to the chain.
blockchain.add_new_block(100)


blockchain.new_transaction("Christine", "Ray", 300)
blockchain.new_transaction("Peter", "Ray", 500)
blockchain.new_transaction("Ray", "Amy", 100)
blockchain.new_transaction("Peter", "Bill", 200)
blockchain.add_new_block(100)

blockchain.print()

Block 1:
Createion time: 2021-04-07 21:03:49
Previous hash: Genesis Block
Hash:0ba1aad51bad1aa325ff00cd24704f008880924575324c3dfd74f4067a81d230

Block 2:
Createion time: 2021-04-07 21:03:49
Previous hash: 0ba1aad51bad1aa325ff00cd24704f008880924575324c3dfd74f4067a81d230
Hash:418067de390918b4d67a4fcaa958b963d6bad481c60d7a855a7c87ea73cb8fd4
{'Sender': 'Ray', 'Recipient': 'Joe', 'Amount(HKD)': 200, 'Datetime': 1617800629.0504622}
{'Sender': 'Peter', 'Recipient': 'Bill', 'Amount(HKD)': 400, 'Datetime': 1617800629.0504622}

Block 3:
Createion time: 2021-04-07 21:03:49
Previous hash: 418067de390918b4d67a4fcaa958b963d6bad481c60d7a855a7c87ea73cb8fd4
Hash:268bfdd8e7c2feae9166d29e943b65521fc76993c31bf58e244cddc75d2c5622
{'Sender': 'Christine', 'Recipient': 'Ray', 'Amount(HKD)': 300, 'Datetime': 1617800629.0504622}
{'Sender': 'Peter', 'Recipient': 'Ray', 'Amount(HKD)': 500, 'Datetime': 1617800629.0504622}
{'Sender': 'Ray', 'Recipient': 'Amy', 'Amount(HKD)': 100, 'Datetime': 1617800629.0504622}
{'S

## Adding difficulty

- Blockchain applications such as Bitcoin set a target for miners to accomplish a certain hash.
To keep the blockchain's growth stable, network **difficulty** is introduced to control the average time for a new block in the blockchain.
- The **difficulty** of a cryptographic puzzle depends on the number of leading zeros in the target. 
- The more leading zeros of the hash value of the target, the more difficult it is to generate a block.

We modify the `add_new_block()` function to introduce the difficulty in our blockchain.


*Orginal code:*

```
    def add_new_block(self, nounce, previous_hash=None):
        block = {
            'index': len(self.blocks) + 1,  # the next value of index
            'timestamp': time.time(),
            'transactions': self.mem_pool,
            'nounce': nounce,
            'previous_hash': previous_hash or self.hash(self.blocks[-1]),
        }
        
        self.mem_pool = [] # clear the pool
        self.blocks.append(block) # append the block to the chain
 ```
 
 We are going to add the followings in the `add_new_block()` function:
 1. Add a variable `difficulty` to represent the difficulty level. It represents the number of leading zeros of the hash value needed.
 2. As we will use the variable `nounce` to generate a different hash value, we take it out from the function signature, which allows the function to keep changing it in a loop.
 3. To help check whether a hash value fulfils the difficulty level, we write a function `check_leading_zeros()`.
 

```
def check_leading_zeros(self, hash_value, difficulty):    
        for i in range(0, difficulty): # extract the character at first difficulty_level position
            if hash_value[i] != "0":
                return False
        return True

```

4. The code for solving the difficulty by bute force are as follows:

```
while not self.check_leading_zeros(hash_value, difficulty):
    nounce = nounce + 1  # try next nounce
    block['nounce'] = nounce
    hash_value = self.hash(block)
```

![nounce](images/lab08_figure1.png)

In [3]:
import time
import json
import hashlib
import datetime

class BlockChain:
    def __init__(self):
        self.blocks = []  # empty chain
        self.mem_pool = [] # empty list
        
        ## add Genesis block
        self.add_new_block("Genesis Block")
        
        
    def new_transaction(self, sender, recipient, amount):
        transaction = {
            "Sender": sender,
            "Recipient": recipient,
            "Amount(HKD)": amount,
            "Datetime": time.time()
        }
        self.mem_pool.append(transaction)
    
    def add_new_block(self, previous_hash=None):
        
        difficulty = 2 # number of leading zeros needed
        
        nounce = 1 # set nounce to 1 first
        
        block = {
            'index': len(self.blocks) + 1,  # the next value of index
            'timestamp': time.time(),
            'transactions': self.mem_pool,
            'nounce': nounce,
            'previous_hash': previous_hash or self.hash(self.blocks[-1]),
        }
        
        hash_value = self.hash(block)
        
        # time the process
        print("Try adding block...")
        start_time = time.time()
        
        while not self.check_leading_zeros(hash_value, difficulty):
            nounce = nounce + 1  # try next nounce
            block['nounce'] = nounce
            hash_value = self.hash(block)

            
        
        
        self.mem_pool = [] # clear the pool
        self.blocks.append(block) # append the block to the chain
        
        
        print("Block added.")
        end_time = time.time()
        print("Total time needed is {} seconds:".format(end_time - start_time))
        
        #return block
    
    def hash(self, block):
        data = json.dumps(block, sort_keys=True)  # convert the json object to string
        return hashlib.sha256(data.encode()).hexdigest()
        
    def print(self):
        for block in self.blocks:
            print("Block {}:".format(block['index']))
            print("Nounce {}".format(block['nounce']))
            print("Createion time: {}".format( self.convert_time(block['timestamp'])))
            print("Previous hash: {}".format(block['previous_hash']))
            print("Hash:{}".format(self.hash(block)))

            for transaction in block["transactions"]:
                print(transaction)

            print("")
        
    def convert_time(self, timestamp):
        return datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")
        
    def check_leading_zeros(self, hash_value, difficulty):    
        for i in range(0, difficulty): # extract the character at first difficulty_level position
            if hash_value[i] != "0":
                return False
        return True

In [4]:
blockchain = BlockChain()

blockchain.new_transaction("Ray", "Joe", 200)
blockchain.new_transaction("Peter", "Bill", 400)
blockchain.add_new_block()


blockchain.new_transaction("Christine", "Ray", 300)
blockchain.new_transaction("Peter", "Ray", 500)
blockchain.new_transaction("Ray", "Amy", 100)
blockchain.new_transaction("Peter", "Bill", 200)
blockchain.add_new_block()


Try adding block...
Block added.
Total time needed is 0.0 seconds:
Try adding block...
Block added.
Total time needed is 0.0 seconds:
Try adding block...
Block added.
Total time needed is 0.001995086669921875 seconds:


In [5]:
# Print the chain out
blockchain.print()

Block 1:
Nounce 4
Createion time: 2021-04-07 21:03:49
Previous hash: Genesis Block
Hash:003a21e1ee316d13542645d368bf125c9c854641795c15ff75cb52ffdc310289

Block 2:
Nounce 28
Createion time: 2021-04-07 21:03:49
Previous hash: 003a21e1ee316d13542645d368bf125c9c854641795c15ff75cb52ffdc310289
Hash:006ae0f426a0ca67c74f376300ebadac2998e2c6fc5f5eca2c8a07c8ac0e9a52
{'Sender': 'Ray', 'Recipient': 'Joe', 'Amount(HKD)': 200, 'Datetime': 1617800629.1033483}
{'Sender': 'Peter', 'Recipient': 'Bill', 'Amount(HKD)': 400, 'Datetime': 1617800629.1043189}

Block 3:
Nounce 129
Createion time: 2021-04-07 21:03:49
Previous hash: 006ae0f426a0ca67c74f376300ebadac2998e2c6fc5f5eca2c8a07c8ac0e9a52
Hash:002fa2e91a54f4db83305f4c13ab1c6cc00eb61457d39d9cb441ae3dcfc4bf53
{'Sender': 'Christine', 'Recipient': 'Ray', 'Amount(HKD)': 300, 'Datetime': 1617800629.1043189}
{'Sender': 'Peter', 'Recipient': 'Ray', 'Amount(HKD)': 500, 'Datetime': 1617800629.1043189}
{'Sender': 'Ray', 'Recipient': 'Amy', 'Amount(HKD)': 100, 'Date


## Exercise 1

Change the difficulty level from 2 to the followings in the above `add_new_block()` function:
1.  `difficulty = 3` 
2.  `difficulty = 4` 
3.  `difficulty = 5` 

How much time is needed for adding a new block in each difficulty? Does the time required has been increased linearly?


## Check for Malicious Changes 

- Blockchain blocks are cryptographically linked to each other by the hash value.
- Each block contains the hash value of its previous block.
- If someone tried to tamper with the data in a specific block, the hash of that particular block and all the subsequent blocks will be changed. That will void the blockchain.

We write the function `is_valid()` to check the hash link.

```
    def is_valid(self):
        length = len(self.blocks)  
        for i in range(0,length):   # loop all the blocks
            if i != 0:  # skip the genesis block
                if self.hash(self.blocks[i-1]) != self.blocks[i]["previous_hash"]:
                    return False
        return True
        
```

In [6]:
import time
import json
import hashlib
import datetime

class BlockChain:
    def __init__(self):
        self.blocks = []  # empty chain
        self.mem_pool = [] # empty list
        
        ## add Genesis block
        self.add_new_block("Genesis Block")
        
        
    def new_transaction(self, sender, recipient, amount):
        transaction = {
            "Sender": sender,
            "Recipient": recipient,
            "Amount(HKD)": amount,
            "Datetime": time.time()
        }
        self.mem_pool.append(transaction)
    
    def add_new_block(self, previous_hash=None):
        
        difficulty = 2 # number of leading zeros needed
        
        nounce = 1 # set nounce to 1 first
        
        block = {
            'index': len(self.blocks) + 1,  # the next value of index
            'timestamp': time.time(),
            'transactions': self.mem_pool,
            'nounce': nounce,
            'previous_hash': previous_hash or self.hash(self.blocks[-1]),
        }
        
        hash_value = self.hash(block)
        
        # time the process
        print("Try adding block...")
        start_time = time.time()
        
        while not self.check_leading_zeros(hash_value, difficulty):
            nounce = nounce + 1  # try next nounce
            block['nounce'] = nounce
            hash_value = self.hash(block)

            
        
        
        self.mem_pool = [] # clear the pool
        self.blocks.append(block) # append the block to the chain
        
        
        print("Block added.")
        end_time = time.time()
        print("Total time needed is {} seconds:".format(end_time - start_time))
        
        # return block
    
    def hash(self, block):
        data = json.dumps(block, sort_keys=True)  # convert the json object to string
        return hashlib.sha256(data.encode()).hexdigest()
        
    def print(self):
        for block in self.blocks:
            print("Block {}:".format(block['index']))
            print("Nounce {}".format(block['nounce']))
            print("Createion time: {}".format( self.convert_time(block['timestamp'])))
            print("Previous hash: {}".format(block['previous_hash']))
            print("Hash:{}".format(self.hash(block)))

            for transaction in block["transactions"]:
                print(transaction)

            print("")
        
    def convert_time(self, timestamp):
        return datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")
        
    def check_leading_zeros(self, hash_value, difficulty):    
        for i in range(0, difficulty): # extract the character at first difficulty_level position
            if hash_value[i] != "0":
                return False
        return True
    
    def is_valid(self):
        length = len(self.blocks)
        for i in range(0,length):
            if i != 0:  # skip the genesis block
                if self.hash(self.blocks[i-1]) != self.blocks[i]["previous_hash"]:
                    return False
        return True

## Test the blockchain

### 1. Build the chain

In [7]:
blockchain = BlockChain()

blockchain.new_transaction("Ray", "Joe", 200)
blockchain.new_transaction("Peter", "Bill", 400)
blockchain.add_new_block()


blockchain.new_transaction("Christine", "Ray", 300)
blockchain.new_transaction("Peter", "Ray", 500)
blockchain.new_transaction("Ray", "Amy", 100)
blockchain.new_transaction("Peter", "Bill", 200)
blockchain.add_new_block()



Try adding block...
Block added.
Total time needed is 0.0040171146392822266 seconds:
Try adding block...
Block added.
Total time needed is 0.005958080291748047 seconds:
Try adding block...
Block added.
Total time needed is 0.0009963512420654297 seconds:


### 2. Print the chain

In [8]:
blockchain.print()

Block 1:
Nounce 457
Createion time: 2021-04-07 21:03:49
Previous hash: Genesis Block
Hash:00692bc691c29c121bca9f99c32e0cca78cf13e2c637701fb8ad2ebe250411da

Block 2:
Nounce 470
Createion time: 2021-04-07 21:03:49
Previous hash: 00692bc691c29c121bca9f99c32e0cca78cf13e2c637701fb8ad2ebe250411da
Hash:003735d6bd99ca12c4b4b78d98bd48c99ed3a87860a8cc360654c01deb45a03b
{'Sender': 'Ray', 'Recipient': 'Joe', 'Amount(HKD)': 200, 'Datetime': 1617800629.1671774}
{'Sender': 'Peter', 'Recipient': 'Bill', 'Amount(HKD)': 400, 'Datetime': 1617800629.1671774}

Block 3:
Nounce 66
Createion time: 2021-04-07 21:03:49
Previous hash: 003735d6bd99ca12c4b4b78d98bd48c99ed3a87860a8cc360654c01deb45a03b
Hash:00a235282c4599558e7dc5ab615547a93a956bc70c72ea4e6581c1c3d261fd06
{'Sender': 'Christine', 'Recipient': 'Ray', 'Amount(HKD)': 300, 'Datetime': 1617800629.1731355}
{'Sender': 'Peter', 'Recipient': 'Ray', 'Amount(HKD)': 500, 'Datetime': 1617800629.1731355}
{'Sender': 'Ray', 'Recipient': 'Amy', 'Amount(HKD)': 100, 'Da

### 3. Check the chain

In [9]:
blockchain.is_valid()

True

### 4. Alter the block data

Now, we change the data in the 2nd transaction by changing the amount from \$400 to \$200.

The blockchain is no longer be valid after the change.

In [10]:
# check the original value
blockchain.blocks[1]['transactions'][1]['Amount(HKD)'] 

400

In [11]:
# Now change a little thing in block 2 
blockchain.blocks[1]['transactions'][1]['Amount(HKD)'] = 200

In [12]:
blockchain.is_valid()

False

### 5. Print the chain again

Now, print the chain out again and check the details.

In [13]:
blockchain.print()

Block 1:
Nounce 457
Createion time: 2021-04-07 21:03:49
Previous hash: Genesis Block
Hash:00692bc691c29c121bca9f99c32e0cca78cf13e2c637701fb8ad2ebe250411da

Block 2:
Nounce 470
Createion time: 2021-04-07 21:03:49
Previous hash: 00692bc691c29c121bca9f99c32e0cca78cf13e2c637701fb8ad2ebe250411da
Hash:7130f142b33116af3cf4aded29bdfc3ed4946e5d87812df5e9d276f3b447edba
{'Sender': 'Ray', 'Recipient': 'Joe', 'Amount(HKD)': 200, 'Datetime': 1617800629.1671774}
{'Sender': 'Peter', 'Recipient': 'Bill', 'Amount(HKD)': 200, 'Datetime': 1617800629.1671774}

Block 3:
Nounce 66
Createion time: 2021-04-07 21:03:49
Previous hash: 003735d6bd99ca12c4b4b78d98bd48c99ed3a87860a8cc360654c01deb45a03b
Hash:00a235282c4599558e7dc5ab615547a93a956bc70c72ea4e6581c1c3d261fd06
{'Sender': 'Christine', 'Recipient': 'Ray', 'Amount(HKD)': 300, 'Datetime': 1617800629.1731355}
{'Sender': 'Peter', 'Recipient': 'Ray', 'Amount(HKD)': 500, 'Datetime': 1617800629.1731355}
{'Sender': 'Ray', 'Recipient': 'Amy', 'Amount(HKD)': 100, 'Da

# End